Esta notebook contiene bloques de código útiles para realizar Q-learning en el entorno "Pendulum"

In [1]:
import numpy as np
from pendulum_env_extended import PendulumEnvExtended
import random 

In [2]:
env = PendulumEnvExtended(render_mode='rgb_array')

Discretización de los estados

In [3]:
x_space = np.linspace(-1, 1, 10)
y_space = np.linspace(-1, 1, 10)
vel_space = np.linspace(-8, 8, 100)
x_space

array([-1.        , -0.77777778, -0.55555556, -0.33333333, -0.11111111,
        0.11111111,  0.33333333,  0.55555556,  0.77777778,  1.        ])

Obtener el estado a partir de la observación

In [4]:
def get_state(obs):
    x, y, vel = obs
    x_bin = np.digitize(x, x_space)
    y_bin = np.digitize(y, y_space)
    vel_bin = np.digitize(vel, vel_space)
    return x_bin, y_bin, vel_bin

In [5]:
state = get_state(np.array([-0.4, 0.2, 0.3]))
state

(3, 6, 52)

Discretización de las acciones

In [6]:
actions = list(np.linspace(-2, 2, 10))
actions

[-2.0,
 -1.5555555555555556,
 -1.1111111111111112,
 -0.6666666666666667,
 -0.22222222222222232,
 0.22222222222222232,
 0.6666666666666665,
 1.1111111111111107,
 1.5555555555555554,
 2.0]

In [7]:
def get_sample_action():
    return random.choice(actions)

Inicilización de la tabla Q

In [8]:
Q = np.zeros((len(x_space) + 1, len(y_space) + 1, len(vel_space) + 1, len(actions)))
Q

array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
    

Obtención de la acción a partir de la tabla Q

In [9]:
def optimal_policy(state, Q):
    action = actions[np.argmax(Q[state])]
    return action

Epsilon-Greedy Policy

In [10]:
def epsilon_greedy_policy(state, Q, epsilon=0.1):
    explore = np.random.binomial(1, epsilon)
    if explore:
        action = get_sample_action()
        print('explore')
    # exploit
    else:
        action = optimal_policy(state, Q)
        print('exploit')
        
    return action

Ejemplo de episodio 

In [11]:
obs,_ = env.reset()
print(obs)
done = False
total_reward = 0
state = get_state(obs)
while not done:
    # Acción del modelo
    action = epsilon_greedy_policy(state, Q, 0.5)
    
    # Indice de la accion en Q
    action_idx = actions.index(action)
    
    # Acción del ambiente
    real_action = np.array([action])
     
    obs, reward, done, _, _ = env.step(real_action)
    next_state = get_state(obs)
    
   # Usar action_idx para actualizar Q
   
    total_reward += reward
    print('->', state, action, reward, obs, done)
    env.render()
print('total_reward', total_reward)

[0.83758765 0.5463029  0.662408  ]
exploit
-> (9, 7, 54) -2.0 -0.3818975963952661 [0.8158778  0.57822436 0.7721352 ] False
explore
-> (9, 7, 54) 1.1111111111111107 -0.4409885710009289 [0.77430886 0.6328079  1.3724701 ] False
explore
-> (9, 7, 54) -0.22222222222222232 -0.6578804641425356 [0.7138181  0.70033115 1.8137428 ] False
explore
-> (9, 7, 54) 1.5555555555555554 -0.9333467975132922 [0.61809635 0.78610235 2.5723245 ] False
explore
-> (9, 7, 54) -0.22222222222222232 -1.4798142884160288 [0.4880816 0.872798  3.128568 ] False
explore
-> (9, 7, 54) -1.5555555555555556 -2.106734810242148 [0.3263114 0.9452623 3.549833 ] False
explore
-> (9, 7, 54) -1.5555555555555556 -2.796179752122054 [0.13075098 0.99141526 4.0254464 ] False
exploit
-> (9, 7, 54) -2.0 -3.6970713039343095 [-0.09219286  0.9957412   4.469008  ] False
exploit
-> (9, 7, 54) -2.0 -4.767172219030381 [-0.3317091  0.9433817  4.915814 ] False
exploit
-> (9, 7, 54) -2.0 -6.064463580306398 [-0.5681715  0.8229102  5.32335  ] False
ex